In [20]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pdfplumber
import textwrap
import pandas as pd

In [23]:
def extract_tables_to_pdf_and_excel(pdf_path, output_pdf_path, excel_output_path):
    # Configurações iniciais para o PDF de saída
    pdf_writer = canvas.Canvas(output_pdf_path, pagesize=letter)
    page_width, page_height = letter
    
    # Armazena DataFrames para exportação para Excel
    dfs = {}

    with pdfplumber.open(pdf_path) as pdf:
        # Itera sobre as páginas do PDF
        for page_number, page in enumerate(pdf.pages):
            # Extrai tabelas da página atual
            tables = page.extract_tables()
            
            for table_index, table in enumerate(tables):
                # Preparando a tabela para o PDF
                num_columns = len(table[0])
                column_width = (page_width - 120) // num_columns
                pdf_writer.drawString(60, page_height - 50, f"Tabela {table_index+1}, Página {page_number+1}")
                y_position = page_height - 70
                
                # Preparando a tabela para o Excel
                df = pd.DataFrame(table[1:], columns=table[0])
                sheet_name = f"Page_{page_number+1}_Table_{table_index+1}"
                dfs[sheet_name] = df
                
                # Desenha a tabela no PDF
                for row in table:
                    x_position = 60
                    max_height = 0
                    for cell in row:
                        if cell:  # Verifica se a célula contém texto
                            text = str(cell)
                            wrapped_text = textwrap.fill(text, width=int(column_width / 6))
                        else:
                            wrapped_text = ""
                        pdf_writer.drawString(x_position, y_position, wrapped_text)
                        x_position += column_width
                        
                        # Calcular a altura necessária para esta célula
                        cell_height = (wrapped_text.count('\n') + 1) * 12
                        max_height = max(max_height, cell_height)
                    
                    y_position -= max_height + 5
                    
                pdf_writer.showPage()
    
    # Salva o PDF
    pdf_writer.save()
    
    # Transcreve nas abas do Excel cada Tabela encontrada e salva do pdf para excel;
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        for sheet_name, df in dfs.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    return dfs  # Retorna os dados para possível uso posterior

# Descomentar para usar
#extract_tables_to_pdf_and_excel("PDF route to be read .pdf file", "Route to store .pdf file", "Route to store Excel .xlsx file")

{'Page_1_Table_1':     NaN DIRETORIA DE AVALIAÇÃO\nDA EDUCAÇÃO SUPERIOR\nDAES   NaN
 0  None                                                     None
 1                                                               
 2  None                                                     None
 3  None                                                     None,
 'Page_11_Table_1':   CONCEITO                                CrITÉrIO DE ANÁLISE
 0        1  As políticas institucionais de ensino, extensã...
 1        2  As políticas institucionais de ensino, extensã...
 2        3  As políticas institucionais de ensino, extensã...
 3        4  As políticas institucionais de ensino, extensã...
 4        5  As políticas institucionais de ensino, extensã...,
 'Page_12_Table_1':   CONCEITO                                CrITÉrIO DE ANÁLISE
 0        1  Os objetivos do curso, constantes no PPC, não ...
 1        2  Os objetivos do curso, constantes no PPC, estã...
 2        3  Os objetivos do curso, constante